In [2]:
import easymore
### load modules
import os
import numpy as np
import xarray as xs
import pandas as pd
import geopandas as gpd
# 
### define input file
input_forcing  = '/scratch/fuaday/ncrb-models/easymore-outputs-merged/ncrbbb_rdrs_1980_2018_v21_wholeVarTimeseries.nc'
output_forcing = '/scratch/fuaday/ncrb-models/easymore-outputs-merged/MESH_input_ncrb_RDRS_v2p1_1980_2018.nc'
input_basin    = '/home/fuaday/scratch/ncrb-models/geofabric-outputs/ncrb-geofabric/ncrb_subbasins.shp'
input_ddb      = '/home/fuaday/scratch/ncrb-models/MESH-ncrb/MESH_drainage_database.nc'

In [3]:
#
### reading input basin
basin = gpd.read_file(input_basin)

In [4]:
db = xs.open_dataset(input_ddb)

In [21]:
db

<xarray.Dataset>
Dimensions:     (subbasin: 32276, NGRU: 17)
Coordinates:
  * subbasin    (subbasin) int64 71006628 71006557 ... 71050687 71004266
Dimensions without coordinates: NGRU
Data variables: (12/13)
    ChnlSlope   (subbasin) float64 ...
    ChnlLength  (subbasin) float64 ...
    Rank        (subbasin) float64 ...
    Next        (subbasin) float64 ...
    GRU         (subbasin, NGRU) float64 ...
    GridArea    (subbasin) float64 ...
    ...          ...
    IAK         (subbasin) int64 ...
    DA          (subbasin) float64 ...
    IWF         (subbasin) int64 ...
    lat         (subbasin) float64 56.47 56.44 56.5 56.5 ... 52.72 52.05 56.93
    lon         (subbasin) float64 -97.87 -97.95 -97.97 ... -108.5 -110.4 -92.7
    crs         int64 ...
Attributes:
    author:       University of Calgary
    license:      GNU General Public License v3 (or any later version)
    purpose:      Create a drainage database .nc file for MESH
    featureType:  point

In [5]:
db.close()

In [9]:
lon = db.variables['lon'].values
lat = db.variables['lat'].values
#segid =  db.variables['seg_id'].values

In [10]:
forc = xs.open_dataset(input_forcing)
forc.close()
lon_ease = forc.variables['longitude'].values
lat_ease = forc.variables['latitude'].values

In [22]:
forc

<xarray.Dataset>
Dimensions:                (time: 120, COMID: 32276)
Coordinates:
  * time                   (time) datetime64[ns] 1986-12-27T13:00:00 ... 1987...
  * COMID                  (COMID) float64 7.1e+07 7.1e+07 ... 7.105e+07
Data variables:
    latitude               (COMID) float64 59.06 59.07 59.03 ... 52.72 52.05
    longitude              (COMID) float64 -101.8 -101.8 ... -108.5 -110.4
    RDRS_v2.1_P_P0_SFC     (time, COMID) float64 ...
    RDRS_v2.1_P_HU_09944   (time, COMID) float64 ...
    RDRS_v2.1_P_TT_09944   (time, COMID) float64 ...
    RDRS_v2.1_P_UVC_09944  (time, COMID) float64 ...
    RDRS_v2.1_A_PR0_SFC    (time, COMID) float64 ...
    RDRS_v2.1_P_FB_SFC     (time, COMID) float64 ...
    RDRS_v2.1_P_FI_SFC     (time, COMID) float64 ...
Attributes:
    CDI:            Climate Data Interface version 2.2.4 (https://mpimet.mpg....
    Conventions:    CF-1.6
    Author:         The data were written by Agent
    License:        GPLv3+; Originla license: No license from original file i...
    History:        Created Thu Apr 11 16:50:19 2024by EASYMORE nc_remapper; ...
    easymore_hash:  915679249cf6cd946e63a09e37b142046bf1651347980b5dd2a56d687...
    Source:         Remapped by EASYMORE nc_remapper from original file: /hom...
    history:        Wed Apr 24 18:52:47 2024: cdo mergetime /scratch/fuaday/n...
    CDO:            Climate Data Operators version 2.2.2 (https://mpimet.mpg....

In [11]:
lon_ease

array([-101.78370301, -101.84916449, -101.92623211, ...,  -96.66232648,
       -108.52132649, -110.41044287])

In [12]:
lon

array([ -97.86783365,  -97.95057772,  -97.97419616, ..., -108.5213268 ,
       -110.41044372,  -92.70304336])

In [18]:
### reorder input forcing
forc_vec = xs.Dataset(
    coords={
        "time": forc['time'].values.copy(),
        "lon": (["subbasin"], lon),
        "lat": (["subbasin"], lat), 
        }
        )

In [27]:
forc_vec

<xarray.Dataset>
Dimensions:                (time: 120, subbasin: 32276)
Coordinates:
  * time                   (time) datetime64[ns] 1986-12-27T13:00:00 ... 1987...
    lon                    (subbasin) float64 -97.87 -97.95 ... -110.4 -92.7
    lat                    (subbasin) float64 56.47 56.44 56.5 ... 52.05 56.93
Dimensions without coordinates: subbasin
Data variables:
    RDRS_v2.1_P_P0_SFC     (subbasin, time) float64 951.1 951.3 ... 923.0 922.2
    RDRS_v2.1_P_HU_09944   (subbasin, time) float64 0.0022 0.002231 ... 0.002429
    RDRS_v2.1_P_TT_09944   (subbasin, time) float64 -6.347 -6.457 ... -4.328
    RDRS_v2.1_A_PR0_SFC    (subbasin, time) float64 0.0 1.41e-06 ... 0.0 0.0
    RDRS_v2.1_P_FB_SFC     (subbasin, time) float64 0.0 0.0 0.0 ... 0.0 0.0 0.0
    RDRS_v2.1_P_FI_SFC     (subbasin, time) float64 206.6 228.9 ... 200.2 200.6
    RDRS_v2.1_P_UVC_09944  (subbasin, time) float64 13.86 13.9 ... 22.65 22.99

In [26]:
# for n in ['pr','hus','wind','ps','ta','rsds','rlds']:	
for n in ['RDRS_v2.1_P_P0_SFC','RDRS_v2.1_P_HU_09944','RDRS_v2.1_P_TT_09944',
          'RDRS_v2.1_A_PR0_SFC','RDRS_v2.1_P_FB_SFC','RDRS_v2.1_P_FI_SFC',
          'RDRS_v2.1_P_UVC_09944']:   
    forc_vec[n] = (("subbasin", "time"), forc[n].values.transpose())
    forc_vec[n].coords["time"]          = forc['time'].values.copy()
    forc_vec[n].coords["lon"]           = (["subbasin"], lon)
    forc_vec[n].coords["lat"]           = (["subbasin"], lat)
    forc_vec[n].attrs["long_name"]      = forc[n].long_name
    forc_vec[n].attrs["units"]          = forc[n].units
    forc_vec[n].attrs["grid_mapping"]   = 'crs'
    forc_vec[n].encoding['coordinates'] = 'time lon lat'